In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
from songs import *
import copy
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('basic midi/track (' + str(i) + ').mid'))

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [3]:
all_songs = np.array([song.notes for song in Songs])

In [4]:
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = all_songs.shape[0]
display_step = 200

# Network Parameters
num_input = 13
timesteps = 128 # timesteps
num_hidden = 100 # hidden layer num of features

# tf Graph input
X = tf.placeholder("float", [batch_size, timesteps, num_input])

# Define weights
w = tf.Variable(tf.random_normal([num_hidden, num_input]))
b = tf.Variable(tf.random_normal([num_input]))

aw = tf.Variable(tf.random_normal([num_hidden]))
ab = tf.Variable(0.0)

class lstm:
    def __init__(self, name, x, num_of_neurons):
        with tf.variable_scope(name):
            self.lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
            self.hidden_state = tf.zeros([batch_size, num_of_neurons])
            self.current_state = tf.zeros([batch_size, num_of_neurons])
            state = self.hidden_state, self.current_state
            self.outputs = []
            for inp in x:
                output, state = self.lstm_cell(inp, state)
                self.outputs.append(output)

_X = tf.unstack(X, timesteps, 1)
first_layer = lstm("fl", _X, num_hidden)
second_layer = lstm("sl", first_layer.outputs, num_hidden)

history = [tf.zeros([1, num_hidden], "float") for i in range(batch_size)]
history_truth = [tf.zeros([1, num_input], "float") for i in range(batch_size)]

checkers = []
for t in range(timesteps):
    checkers.append([])
    
    for i in range(batch_size):
        distance = tf.reduce_sum(tf.abs(tf.add(history[i], tf.negative(second_layer.outputs[t][i]))), reduction_indices=1)
        checkers[t].append(history_truth[i][tf.arg_min(distance, 0)])
        
        history[i] = tf.concat([history[i], tf.expand_dims(second_layer.outputs[t][i], 0)], axis=0)
        history_truth[i] = tf.concat([history_truth[i], tf.expand_dims(_X[t][i], 0)], axis=0)
        
repeats = tf.convert_to_tensor(checkers)

side_layer = lstm("sidel", first_layer.outputs, num_hidden)
a = tf.expand_dims(tf.sigmoid(tf.matmul(side_layer.outputs[i], tf.expand_dims(aw, 1)) + ab), 0)

logits = a * tf.sigmoid(tf.matmul(second_layer.outputs[i], w) + b) + (1 - a) * checkers

# Define loss and optimizer
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=X))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
loss_op = -tf.reduce_mean(tf.multiply(tf.convert_to_tensor(_X), tf.log(logits)) +
                         tf.multiply(1. - tf.convert_to_tensor(_X), tf.log(1. - logits)))
#train_op = optimizer.minimize(loss_op)
train_op = tf.contrib.layers.optimize_loss(
    loss_op, tf.contrib.framework.get_global_step(), optimizer=optimizer,
    learning_rate=learning_rate)

# Evaluate model (with test logits, for dropout to be disabled)
#correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

TypeError: Input 'begin' of 'StridedSliceGrad' Op has type int64 that does not match type int32 of argument 'shape'.

In [ ]:
# Start training
sess = tf.Session()
# Run the initializer
sess.run(init)

for step in range(0, training_steps):
    batch_x = all_songs#mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    
    # Run optimization op (backprop)
    sess.run(train_op, feed_dict={X: batch_x})
    if step % display_step == 0:
        # Calculate batch loss and accuracy
        L = sess.run(loss_op, feed_dict={X: batch_x})
        print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(L))

print("Optimization Finished!")

In [111]:
all_songs = np.array([song.notes for song in Songs])
batch_x = all_songs
batch_x = batch_x.reshape((69, timesteps, num_input))

In [112]:
outp = np.array(sess.run(logits, feed_dict={X: batch_x}))

In [113]:
first_song = outp[:, 0, :]

In [114]:
first_song[all_songs[0][:32] == 1]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 128 but corresponding boolean dimension is 32
  """Entry point for launching an IPython kernel.


array([ 0.35305798,  0.40715349,  0.43449512,  0.51579624,  0.4916901 ,
        0.57061625,  0.56180316,  0.54283768,  0.56193489,  0.53043222,
        0.55983573,  0.52543014,  0.53507745,  0.51688242], dtype=float32)

In [115]:
first_song[all_songs[0][32:64] == 1]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 128 but corresponding boolean dimension is 32
  """Entry point for launching an IPython kernel.


array([ 0.35305798,  0.40715349,  0.43449512,  0.51579624,  0.4916901 ,
        0.57061625,  0.56180316,  0.54283768,  0.56193489,  0.53043222,
        0.55983573,  0.52543014,  0.53507745], dtype=float32)

Проблема: LSTM ни фига не запоминает

In [116]:
new_songs = [MySong() for i in range(69)]

In [117]:
for i in range(128):
    outp = np.array(sess.run(logits, feed_dict={X: batch_x}))
    for s in range(69):
        next_one = outp[i, s, :]
            
        next_one[next_one < 0.65] = 0
        next_one[next_one != 0] = 1
        
        if i >= 8:
            batch_x[s, i] = next_one
            new_songs[s].add(next_one)
        else:
            new_songs[s].add(batch_x[s, i])

In [118]:
new_songs[0].play()